In [43]:
import pandas as pd

In [44]:
leafly = pd.read_json('../data/leafly.json')
leafly.shape

(3447, 37)

# Data Wrangling

In [45]:
# Drop nans from leafly

leafly = leafly.dropna()
leafly.shape

(311, 37)

In [46]:
# Combine feelings into one column for one hot encoding

leafly['feelings'] = leafly['feeling_1'] + ',' + leafly['feeling_2'] + ',' + leafly['feeling_3'] + ',' + leafly['feeling_4'] + ',' + leafly['feeling_5']

In [47]:
# Check feelings

leafly['feelings']

0       Happy,Euphoric,Creative,Relaxed,Uplifted
42        Happy,Relaxed,Euphoric,Hungry,Uplifted
45         Relaxed,Happy,Euphoric,Hungry,Focused
47       Happy,Relaxed,Sleepy,Talkative,Uplifted
52          Relaxed,Happy,Sleepy,Euphoric,Hungry
                          ...                   
3341      Relaxed,Happy,Euphoric,Sleepy,Uplifted
3364    Relaxed,Euphoric,Uplifted,Happy,Creative
3370      Relaxed,Euphoric,Happy,Uplifted,Sleepy
3415      Happy,Relaxed,Euphoric,Uplifted,Hungry
3434    Happy,Energetic,Relaxed,Focused,Uplifted
Name: feelings, Length: 311, dtype: object

In [48]:
# Combine helps into one column for one hot encoding

leafly['helps'] = leafly['helps_1'] + ',' + leafly['helps_2'] + ',' + leafly['helps_3'] + ',' + leafly['helps_4'] + ',' + leafly['helps_5']

In [49]:
# Check helps

leafly['helps']

0                 Stress,Pain,Anxiety,Depression,Nausea
42      Depression,Stress,Pain,Anxiety,Lack of appetite
45                Stress,Anxiety,Nausea,Pain,Depression
47             Pain,Stress,Anxiety,Depression,Headaches
52              Stress,Pain,Insomnia,Anxiety,Depression
                             ...                       
3341            Stress,Anxiety,Pain,Depression,Insomnia
3364             Pain,Stress,Depression,Anxiety,Fatigue
3370            Stress,Anxiety,Depression,Insomnia,Pain
3415            Stress,Pain,Anxiety,Depression,Insomnia
3434             Stress,Depression,Pain,Fatigue,Anxiety
Name: helps, Length: 311, dtype: object

In [50]:
# Combine feelings and helps columnbs

leafly["feelings_helps"] = leafly['feelings'] + ',' + leafly['helps'] 

In [51]:
# Check feelings_helps

leafly['feelings_helps']

0       Happy,Euphoric,Creative,Relaxed,Uplifted,Stres...
42      Happy,Relaxed,Euphoric,Hungry,Uplifted,Depress...
45      Relaxed,Happy,Euphoric,Hungry,Focused,Stress,A...
47      Happy,Relaxed,Sleepy,Talkative,Uplifted,Pain,S...
52      Relaxed,Happy,Sleepy,Euphoric,Hungry,Stress,Pa...
                              ...                        
3341    Relaxed,Happy,Euphoric,Sleepy,Uplifted,Stress,...
3364    Relaxed,Euphoric,Uplifted,Happy,Creative,Pain,...
3370    Relaxed,Euphoric,Happy,Uplifted,Sleepy,Stress,...
3415    Happy,Relaxed,Euphoric,Uplifted,Hungry,Stress,...
3434    Happy,Energetic,Relaxed,Focused,Uplifted,Stres...
Name: feelings_helps, Length: 311, dtype: object

In [52]:
# Remove commas and add spaces

leafly['feelings'] = leafly['feelings'].str.replace(',', ' ')
leafly['helps']= leafly['helps'].str.replace(',',' ')

In [53]:
# Combine all text columns into one

leafly['feelings_helps_description'] = leafly["feelings"] + ' ' + leafly["helps"] + ' ' + leafly['description']

In [54]:
# Verify new column

leafly['feelings_helps_description']

0       Happy Euphoric Creative Relaxed Uplifted Stres...
42      Happy Relaxed Euphoric Hungry Uplifted Depress...
45      Relaxed Happy Euphoric Hungry Focused Stress A...
47      Happy Relaxed Sleepy Talkative Uplifted Pain S...
52      Relaxed Happy Sleepy Euphoric Hungry Stress Pa...
                              ...                        
3341    Relaxed Happy Euphoric Sleepy Uplifted Stress ...
3364    Relaxed Euphoric Uplifted Happy Creative Pain ...
3370    Relaxed Euphoric Happy Uplifted Sleepy Stress ...
3415    Happy Relaxed Euphoric Uplifted Hungry Stress ...
3434    Happy Energetic Relaxed Focused Uplifted Stres...
Name: feelings_helps_description, Length: 311, dtype: object

In [55]:
import pickle

# Load nearest neighbors model
model_pkl = open('../pickles/nn_model.pkl', 'rb')
NN_model = pickle.load(model_pkl)
print ("Loaded model :: ", NN_model)

# load tfidf transformer
tfidf_pkl = open('../pickles/tfidf.pkl' , 'rb')
tfidf_model = pickle.load(tfidf_pkl)
print ("Loaded model :: ", tfidf_model)  # print to verify

Loaded model ::  NearestNeighbors(algorithm='kd_tree', n_neighbors=4)
Loaded model ::  TfidfVectorizer(max_features=2000, ngram_range=(1, 2), stop_words='english')


In [56]:
# Demo Recommend Function from DS

import json

def recommend(user_input):
    temp_df = NN_model.kneighbors(tfidf_model.transform([user_input]).todense())[1]

    for i in range(4):
        info = leafly.iloc[temp_df[0][i]]['strain']
        info_aka = leafly.iloc[temp_df[0][i]]['aka']
        info_type = leafly.iloc[temp_df[0][i]]['type']
        info_rating = leafly.iloc[temp_df[0][i]]['rating']
        info_num_reviews= leafly.iloc[temp_df[0][i]]['num_reviews']
        info_feelings = leafly.iloc[temp_df[0][i]]['feelings']
        info_helps = leafly.iloc[temp_df[0][i]]['helps']
        info_description = leafly.iloc[temp_df[0][i]]['description']
        
        print(json.dumps(info))
        print(json.dumps(info_aka))
        print(json.dumps(info_type))
        print(json.dumps(info_rating))
        print(json.dumps(info_num_reviews))
        print(json.dumps(info_feelings))
        print(json.dumps(info_helps))
        print(json.dumps(info_description))

In [57]:
recommend('Relaxed Happy Euphoric Uplifted Hungry')

"Cherry Cola"
"Cherry Kola"
"Indica"
4.7
61.0
"Relaxed Euphoric Happy Uplifted Hungry"
"Stress Pain Anxiety Depression Lack of appetite"
"Cherry Kola is an indica strain that competed in the 2014 L.A. Cannabis Cup."
"Hindu Skunk"
"Dragonite, Skunk Kush, Hindu Kush Skunk"
"Indica"
4.1
107.0
"Euphoric Relaxed Happy Uplifted Hungry"
"Stress Anxiety Pain Depression Insomnia"
"Also known as Skunk Kush,\u00a0Hindu Skunk is a cross of Hindu Kush and Skunk #1.\u00a0The taste of the smoke is smooth and provides a strong and very mellow high. \u00a0"
"Cali Kush"
"California Kush, Kali Kush"
"Hybrid"
4.1
242.0
"Relaxed Happy Euphoric Uplifted Hungry"
"Stress Depression Anxiety Pain Headaches"
"Cali Kush must have been a no-brainer when it came time to name this strain given the popularity of Kush varieties in The Golden State. Its buds are large and dense with few leaves and not many pistils. The flowers will be light green shot through with icy blue and packed with trichomes throughout. Cali Kus

# Pickle minimal data needed for output

In [58]:
min_data = leafly[['strain', 'type', 'feeling_1', 'feeling_2', 'feeling_3', 'feeling_4','feeling_5', 'helps_1', 'helps_2', 'helps_3', 'helps_4', 'helps_5', 'description']].to_dict('records')

with open('../pickles/min_data.pkl', 'wb') as data_pkl:
    pickle.dump(min_data, data_pkl)

# Code for use in api

need to adjust paths

In [59]:
import pickle

with open('../pickles/nn_model.pkl', 'rb') as nn_pkl:
    model = pickle.load(nn_pkl)

with open('../pickles/tfidf.pkl', 'rb') as tfidf_pkl:
    tfidf = pickle.load(tfidf_pkl)

with open('../pickles/min_data.pkl', 'rb') as data_pkl:
    data = pickle.load(data_pkl)

def get_recommendations(user_input:str, num: int = 4):
    neighbors = model.kneighbors(
        tfidf.transform([user_input]).todense(),
        n_neighbors=num, return_distance=False
    )
    results = []
    for index in neighbors[0]:
        results.append({
            'strain': data[index]['strain'],
            'strain_type': data[index]['type'],
            'description': data[index]['description'],
            'effects': [data[index][f'feeling_{i}'] for i in range(1,6)],
            'helps': [data[index][f'helps_{i}'] for i in range(1,6)],
        })
    return results

In [60]:
get_recommendations('Relaxed Happy Euphoric Uplifted Hungry')

[{'strain': 'Cherry Cola',
  'strain_type': 'Indica',
  'description': 'Cherry Kola is an indica strain that competed in the 2014 L.A. Cannabis Cup.',
  'effects': ['Relaxed', 'Euphoric', 'Happy', 'Uplifted', 'Hungry'],
  'helps': ['Stress', 'Pain', 'Anxiety', 'Depression', 'Lack of appetite']},
 {'strain': 'Hindu Skunk',
  'strain_type': 'Indica',
  'description': 'Also known as Skunk Kush,\xa0Hindu Skunk is a cross of Hindu Kush and Skunk #1.\xa0The taste of the smoke is smooth and provides a strong and very mellow high. \xa0',
  'effects': ['Euphoric', 'Relaxed', 'Happy', 'Uplifted', 'Hungry'],
  'helps': ['Stress', 'Anxiety', 'Pain', 'Depression', 'Insomnia']},
 {'strain': 'Cali Kush',
  'strain_type': 'Hybrid',
  'description': 'Cali Kush must have been a no-brainer when it came time to name this strain given the popularity of Kush varieties in The Golden State. Its buds are large and dense with few leaves and not many pistils. The flowers will be light green shot through with icy